## Model Selection and Modeling

- [Linear Regression: OLS](#lm)
- [Random Forest Regression](#rfr)
- [Light Gradient Boosting Model With Bayesian Optimization](#lgbm)

### Import Necessary Modules and Datasets

In [1]:
# Import tools to get datasets
from zipfile import ZipFile
import urllib
import requests
import io

# Import modules for data reading and plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Import all for model selections and modelling
from bayes_opt import BayesianOptimization
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
import time

In [2]:
start = time.time() # Timer starts

Xtrainurl = 'https://raw.githubusercontent.com/jonahwinninghoff/Springboard_Capstone_Project/main/Assets/X_train'
Xvalidurl = 'https://raw.githubusercontent.com/jonahwinninghoff/Springboard_Capstone_Project/main/Assets/X_valid'
Xtestiurl = 'https://raw.githubusercontent.com/jonahwinninghoff/Springboard_Capstone_Project/main/Assets/X_test'

ytrainurl = 'https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/y_train?raw=true'
yvalidurl = 'https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/y_valid?raw=true'
ytestiurl = 'https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/y_test?raw=true'

In [3]:
# Read X_train dataset
url = urllib.request.urlopen(Xtrainurl)
file = io.BytesIO(url.read())
X_train = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)

# Read X_valid dataset
url = urllib.request.urlopen(Xvalidurl)
file = io.BytesIO(url.read())
X_valid = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)

# Read X_test dataset
url = urllib.request.urlopen(Xtestiurl)
file = io.BytesIO(url.read())
X_test = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)

In [4]:
# Read y_train dataset
response = requests.get(ytrainurl)
response.raise_for_status()
y_train = np.load(io.BytesIO(response.content))

# Read y_valid dataset
response = requests.get(yvalidurl)
response.raise_for_status()
y_valid = np.load(io.BytesIO(response.content))

# Read y_test dataset
response = requests.get(ytestiurl)
response.raise_for_status()
y_test = np.load(io.BytesIO(response.content))

In [5]:
# Read X_TFIDF_train
url = urllib.request.urlopen('https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/X_TFIDF_train.zip?raw=true')
unzipfile = ZipFile(io.BytesIO(url.read()))
file = unzipfile.open("X_TFIDF_train")
X_TFIDF_train = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)
file.close()

# Read X_TFIDF_valid
url = urllib.request.urlopen('https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/X_TFIDF_valid.zip?raw=true')
unzipfile = ZipFile(io.BytesIO(url.read()))
file = unzipfile.open("X_TFIDF_valid")
X_TFIDF_valid = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)
file.close()

# Read X_TFIDF_test
url = urllib.request.urlopen('https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/X_TFIDF_test.zip?raw=true')
unzipfile = ZipFile(io.BytesIO(url.read()))
file = unzipfile.open("X_TFIDF_test")
X_TFIDF_test = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)
file.close()

### Linear Regression: OLS <a id ='lm'></a>

In [6]:
X_TFIDF_train_wconstant = sm.add_constant(X_TFIDF_train)
model = sm.OLS(y_train,X_TFIDF_train_wconstant)
TFIDF_fitted_lm = model.fit()

display(TFIDF_fitted_lm.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.909
Model:                            OLS   Adj. R-squared:                  0.909
Method:                 Least Squares   F-statistic:                 8.340e+04
Date:                Fri, 03 Sep 2021   Prob (F-statistic):               0.00
Time:                        13:36:25   Log-Likelihood:             1.1004e+05
No. Observations:              141159   AIC:                        -2.200e+05
Df Residuals:                  141141   BIC:                        -2.199e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             6.1332      0.346     17.721      0.000       5.455       6.812
ability       -1.745e+10   2.37e+10     -0.737      0.461   -6.39e+10     2.9e+10
activities    -2.058e+10    5.1e+10     -0.403      0.687   -1.21e+11    7.94e+10
antianxiety    4.197e+10      3e+10      1.401      0.161   -1.67e+10    1.01e+11
antipsychotic  1.241e+10   3.49e+10      0.356      0.722    -5.6e+10    8.08e+10
appropriately -1.489e+10   2.41e+10     -0.618      0.537   -6.22e+10    3.24e+10
assessed      -6.569e+09   2.66e+10     -0.247      0.805   -5.86e+10    4.55e+10
bladder       -5.572e+09   1.39e+11     -0.040      0.968   -2.79e+11    2.68e+11
bowels         1.404e+11    7.8e+10      1.799      0.072   -1.26e+10    2.93e+11
catheter       3.022e+11   1.09e+11      2.766      0.006    8.81e+10    5.16e+11
control        1.711e+11   9.85e+10      1.736      0.083   -2.21e+10    3.64e+11
daily          2.035e+10   1.85e+10      1.102      0.271   -1.58e+10    5.65e+10
depressive      4.38e+10   1.02e+11      0.430      0.668   -1.56e+11    2.44e+11
experiencing   2.383e+10   3.63e+10      0.657      0.511   -4.72e+10    9.49e+10
falls           1.41e+10   3.55e+10      0.397      0.691   -5.55e+10    8.37e+10
function       2.847e+10   1.23e+11      0.231      0.818   -2.13e+11     2.7e+11
given          2.843e+10   3.48e+10      0.816      0.414   -3.98e+10    9.67e+10
help            7.59e+09   1.33e+10      0.569      0.569   -1.85e+10    3.37e+10
high           5.732e+11   1.78e+11      3.228      0.001    2.25e+11    9.21e+11
hypnotic      -3.124e+10   4.66e+10     -0.670      0.503   -1.23e+11    6.02e+10
improvements  -1.077e+10   1.16e+11     -0.093      0.926   -2.37e+11    2.16e+11
increased      4.964e+09   1.31e+10      0.379      0.705   -2.07e+10    3.06e+10
independently   1.65e+10   1.28e+10      1.288      0.198    -8.6e+09    4.16e+10
infection     -2.038e+11   7.43e+10     -2.742      0.006   -3.49e+11   -5.81e+10
influenza     -4.506e+09    2.3e+10     -0.196      0.845   -4.96e+10    4.06e+10
injury        -9.604e+09   3.24e+10     -0.296      0.767   -7.31e+10    5.39e+10
inserted      -1.405e+11   6.67e+10     -2.108      0.035   -2.71e+11   -9.86e+09
left          -1.397e+11   6.67e+10     -2.095      0.036    -2.7e+11   -9.03e+09
long          -5.566e+10   3.95e+10     -1.410      0.158   -1.33e+11    2.17e+10
lose          -3.698e+10   4.61e+10     -0.803      0.422   -1.27e+11    5.33e+10
low           -7.767e+10   1.01e+11     -0.766      0.444   -2.76e+11    1.21e+11
major         -1.017e+10   3.35e+10     -0.304      0.761   -7.57e+10    5.54e+10
medication     5.234e+09   2.31e+10      0.227      0.821      -4e+10    5.05e+10
need           6.132e+09   1.32e+10      0.466      0.641   -1.96e+10    3.19e+10
newly            -2.1122      0.118    -17.968   

In [7]:
X_train_wconstant = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train_wconstant)
fitted_lm = model.fit()

display(fitted_lm.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                 6.464e+04
Date:                Fri, 03 Sep 2021   Prob (F-statistic):               0.00
Time:                        13:36:25   Log-Likelihood:                 24551.
No. Observations:              141159   AIC:                        -4.909e+04
Df Residuals:                  141153   BIC:                        -4.903e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.1596      0.007     23.924      0.000       0.146       0.173
cosine median     -0.0452      0.002    -19.139      0.000      -0.050      -0.041
cosine mean        0.7183      0.015     47.088      0.000       0.688       0.748
cosine minimum    -0.1735      0.009    -20.246      0.000      -0.190      -0.157
cosine maximum     0.6937      0.003    198.503      0.000       0.687       0.701
cosine std        -0.6243      0.035    -18.031      0.000      -0.692      -0.556
==============================================================================
Omnibus:                    16213.742   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28286.985
Skew:                           0.786   Prob(JB):                         0.00
Kurtosis:                       4.530   Cond. No.                         80.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The OLS with TFIDF vectorization shows that a few variables are statistically significant at two-sided 5% alpha level. The Akaike and Bayesian Information Criterions (AIC and BIC) are −0.98 and −0.977. The R-squared score is 90.9%. The adjusted R-squared score shows that the number of parameters has little impact on the percentage of the target variable variation that this model explains. When the number of features reduces using cosine similarities, each variable is satistically significant at two-sided 1% alpha level, and AIC and BIC are lower. However, the R-squared score is 69.9, and the adjusted R-squared score is same. 

The kurtosis score lowers but it is still leptokurtotic. The skewness for OLS with TFIDF vectorization is extremely negative. When the cosine similarites are applied, the skewness shifts to postive or right skewness.

In [8]:
X_valid_wconstant = sm.add_constant(X_valid)
y_predicted = fitted_lm.predict(X_valid_wconstant)

X_TFIDF_valid_wconstant = sm.add_constant(X_TFIDF_valid)
y_TFIDF_predicted = TFIDF_fitted_lm.predict(X_TFIDF_valid_wconstant)

In [9]:
def r2_calculator(predict, true):
    merged = pd.concat([pd.DataFrame(list(predict)).rename(columns={0:'predicted'}), 
           pd.DataFrame(list(true)).rename(columns={0:'true'})],axis=1)
    
    # 1 - (predict - true)^2/(true - mean(true))^2 = 1 - RSS/TSS = r_2
    r2 = 1 - sum((merged['predicted'] - merged['true'])**2)/sum((merged['true'] - np.mean(merged['true']))**2)
    
    return r2

def adjusted_r2_calculator(predict, true):
    r2 = r2_calculator(predict, true) # r_2
    n = len(true)                     # number of rows
    k = len(fitted_lm.params)         # number of parameters
    
    adj_r2 = 1 - ((1-r2)*(n-1))/(n-k-1)    # 1 - [(1 - r_2)*(n - 1)/(n - k - 1)] = adjusted r^2
    
    return adj_r2

def mae_and_mse_calculator(predict, true):
    merged = pd.concat([pd.DataFrame(list(predict)).rename(columns={0:'predicted'}), 
           pd.DataFrame(list(true)).rename(columns={0:'true'})],axis=1)
    mae = sum(np.abs(merged['true'] - merged['predicted']))/len(true) # 1/n ∑ |true - predict|   = MAE
    mse = sum((merged['true'] - merged['predicted'])**2)/len(true)    # 1/n ∑ (true - predict)^2 = MSE
    
    return mae, mse

In [10]:
print('TFIDF Validation Set')
print('R2: '+str(round(r2_calculator(y_TFIDF_predicted, y_valid),4)))
print('Adjusted R2: '+str(round(adjusted_r2_calculator(y_TFIDF_predicted,y_valid),4)))
print('MAE: '+ str(round(mae_and_mse_calculator(y_TFIDF_predicted,y_valid)[0],4)))
print('RMSE: '+ str(round(mae_and_mse_calculator(y_TFIDF_predicted,y_valid)[1]**0.5,4)))

TFIDF Validation Set
R2: 0.9091
Adjusted R2: 0.9091
MAE: 0.0687
RMSE: 0.1108


In [11]:
print('Cos Similarity Validation Set')
print('R2: '+str(round(r2_calculator(y_predicted, y_valid),4)))
print('Adjusted R2: '+str(round(adjusted_r2_calculator(y_predicted,y_valid),4)))
print('MAE: '+ str(round(mae_and_mse_calculator(y_predicted,y_valid)[0],4)))
print('RMSE: '+ str(round(mae_and_mse_calculator(y_predicted,y_valid)[1]**0.5,4)))

Cos Similarity Validation Set
R2: 0.6928
Adjusted R2: 0.6928
MAE: 0.1531
RMSE: 0.2036


When the model with TFIDF vectorization is being generalized using validation set, it is already at highest accuracy and lowest loss. The model with cosine similarities still performs well when it generalizes. The problem with TFIDF vectorization is that the number of parameters is too wide.

## Random Forest Regression <a id='rfr'></a>

In [12]:
TFIDF_rfr_model = RandomForestRegressor()
TFIDF_rfr_model.fit(X_TFIDF_train, y_train.ravel())

rfr_model = RandomForestRegressor()
rfr_model.fit(X_train, y_train.ravel())

RandomForestRegressor()

In [13]:
print('TFIDF Similarity Validation Set')
print('R2: ' + str(round(r2_calculator(TFIDF_rfr_model.predict(X_TFIDF_valid),y_valid),4)))
print('Adjusted R2: ' + str(round(adjusted_r2_calculator(TFIDF_rfr_model.predict(X_TFIDF_valid),y_valid),4)))
print('MAE: ' + str(round(mae_and_mse_calculator(TFIDF_rfr_model.predict(X_TFIDF_valid),y_valid)[0],4)))
print('RMSE: ' + str(round(mae_and_mse_calculator(TFIDF_rfr_model.predict(X_TFIDF_valid),y_valid)[1]**0.5,4)))

TFIDF Similarity Validation Set
R2: 0.9091
Adjusted R2: 0.9091
MAE: 0.0687
RMSE: 0.1107


In [14]:
print('Cos Similarity Validation Set')
print('R2: ' + str(round(r2_calculator(rfr_model.predict(X_valid),y_valid),4)))
print('Adjusted R2: ' + str(round(adjusted_r2_calculator(rfr_model.predict(X_valid),y_valid),4)))
print('MAE: ' + str(round(mae_and_mse_calculator(rfr_model.predict(X_valid),y_valid)[0],4)))
print('RMSE: ' + str(round(mae_and_mse_calculator(rfr_model.predict(X_valid),y_valid)[1]**0.5,4)))

Cos Similarity Validation Set
R2: 0.9091
Adjusted R2: 0.9091
MAE: 0.0687
RMSE: 0.1107


As a result, the random forest regressor proves to be capable of performing the model at highest level even if the number of parameters reduce using cosine similarities. 

## Light Gradient Boosting Model With Bayesian Optimization <a id='lgbm'></a>

In [15]:
def lgbm_eval(lambda_l2,lambda_l1,max_depth,learning_rate,n_estimators):
    lgbm_model = LGBMRegressor(lambda_l2 = lambda_l2, lambda_l1 = lambda_l1, 
                               max_depth = int(round(max_depth,0)),
                               learning_rate = learning_rate, 
                               n_estimators = int(round(n_estimators,0)))
    
    lgbm_model.fit(X_train, y_train.ravel())
    return r2_calculator(lgbm_model.predict(X_valid),y_valid)

In [16]:
lgbmBO = BayesianOptimization(lgbm_eval, {'lambda_l2':(0, 0.5),
                                          'lambda_l1':(0,0.5),
                                          'max_depth':(-1,6),
                                          'learning_rate':(0.01,0.5),
                                          'n_estimators':(10,200)})

lgbmBO.maximize(n_iter=20, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------------------
|  1        |  0.9091   |  0.09487  |  0.1745   |  0.2014   | -0.8501   |  80.65    |
|  2        |  0.9091   |  0.3941   |  0.1887   |  0.1184   |  2.559    |  104.9    |
|  3        |  0.9091   |  0.001538 |  0.2456   |  0.4179   |  0.2545   |  80.02    |
|  4        |  0.9091   |  0.2373   |  0.1865   |  0.3677   |  4.104    |  197.4    |
|  5        |  0.9091   |  0.05774  |  0.07863  |  0.1766   |  4.936    |  44.38    |
|  6        |  0.9091   |  0.3401   |  0.1508   |  0.1808   |  4.572    |  39.32    |
|  7        |  0.9087   |  0.281    |  0.4214   |  0.1559   |  3.108    |  33.88    |
|  8        |  0.9091   |  0.3007   |  0.1272   |  0.185    | -0.5646   |  43.37    |
|  9        |  0.9091   |  0.1609   |  0.3068   |  0.2906   |  0.1803   |  49.78    |
|  10       |  0.9091   |  0.4575   |  0.487    |  0.1

In [17]:
def TFIDF_lgbm_eval(lambda_l2,lambda_l1,max_depth,learning_rate,n_estimators):
    lgbm_model = LGBMRegressor(lambda_l2 = lambda_l2, lambda_l1 = lambda_l1, 
                               max_depth = int(round(max_depth,0)),
                               learning_rate = learning_rate, 
                               n_estimators = int(round(n_estimators,0)))
    
    lgbm_model.fit(X_TFIDF_train, y_train.ravel())
    return r2_calculator(lgbm_model.predict(X_TFIDF_valid),y_valid)

In [18]:
TFIDF_lgbmBO = BayesianOptimization(TFIDF_lgbm_eval, {'lambda_l2':(0, 0.5),
                                          'lambda_l1':(0,0.5),
                                          'max_depth':(-1,6),
                                          'learning_rate':(0.01,0.5),
                                          'n_estimators':(10,200)})

TFIDF_lgbmBO.maximize(n_iter=20, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------------------
|  1        |  0.9091   |  0.4742   |  0.0881   |  0.259    |  4.581    |  135.8    |
|  2        |  0.906    |  0.3256   |  0.3623   |  0.3262   |  0.7226   |  61.46    |
|  3        |  0.9091   |  0.4992   |  0.1555   |  0.4905   |  5.784    |  136.3    |
|  4        |  0.9082   |  0.4417   |  0.1758   |  0.01701  | -0.8859   |  199.8    |
|  5        |  0.8736   |  0.0      |  0.2158   |  0.01     | -1.0      |  161.1    |
|  6        |  0.9091   |  0.5      |  0.4773   |  0.5      |  6.0      |  115.1    |
|  7        |  0.7605   |  0.0      |  0.0      |  0.01     | -1.0      |  89.65    |
|  8        |  0.9091   |  0.5      |  0.5      |  0.5      |  2.242    |  40.88    |
|  9        |  0.9091   |  0.05308  |  0.02874  |  0.3753   | -0.9437   |  14.95    |
|  10       |  0.9091   |  0.0      |  0.5      |  0.5

In [19]:
print(lgbmBO.max['params'])
print(TFIDF_lgbmBO.max['params'])

{'lambda_l1': 0.340093247157719, 'lambda_l2': 0.15080299601720903, 'learning_rate': 0.1808274612032137, 'max_depth': 4.572273005043445, 'n_estimators': 39.323025009117075}
{'lambda_l1': 0.053083311782776144, 'lambda_l2': 0.028736780795506844, 'learning_rate': 0.3752551731221079, 'max_depth': -0.9437298530160639, 'n_estimators': 14.950985278511745}


In [20]:
lgbm_model = LGBMRegressor(lambda_l2 = lgbmBO.max['params']['lambda_l1'],
                           lambda_l1 = lgbmBO.max['params']['lambda_l2'],
                           max_depth = int(round(lgbmBO.max['params']['max_depth'],0)),
                           learning_rate = lgbmBO.max['params']['learning_rate'],
                           n_estimators = int(round(lgbmBO.max['params']['n_estimators'],0)))
lgbm_model.fit(X_train, y_train.ravel())

LGBMRegressor(lambda_l1=0.15080299601720903, lambda_l2=0.340093247157719,
              learning_rate=0.1808274612032137, max_depth=5, n_estimators=39)

In [21]:
TFIDF_lgbm_model = LGBMRegressor(lambda_l2 = TFIDF_lgbmBO.max['params']['lambda_l1'],
                           lambda_l1 = TFIDF_lgbmBO.max['params']['lambda_l2'],
                           max_depth = int(round(TFIDF_lgbmBO.max['params']['max_depth'],0)),
                           learning_rate = TFIDF_lgbmBO.max['params']['learning_rate'],
                           n_estimators = int(round(TFIDF_lgbmBO.max['params']['n_estimators'],0)))
TFIDF_lgbm_model.fit(X_TFIDF_train, y_train.ravel())

LGBMRegressor(lambda_l1=0.028736780795506844, lambda_l2=0.053083311782776144,
              learning_rate=0.3752551731221079, n_estimators=15)

In [22]:
print('TFIDF Validation Set')
print('R2: ' + str(round(r2_calculator(TFIDF_lgbm_model.predict(X_TFIDF_valid),y_valid),4)))
print('Adjusted R2: ' + str(round(adjusted_r2_calculator(TFIDF_lgbm_model.predict(X_TFIDF_valid),y_valid),4)))
print('MAE: ' + str(round(mae_and_mse_calculator(TFIDF_lgbm_model.predict(X_TFIDF_valid),y_valid)[0],4)))
print('RMSE: ' + str(round(mae_and_mse_calculator(TFIDF_lgbm_model.predict(X_TFIDF_valid),y_valid)[1]**0.5,4)))

TFIDF Validation Set
R2: 0.9091
Adjusted R2: 0.9091
MAE: 0.0688
RMSE: 0.1107


In [23]:
print('Cosine Similarity Validation Set')
print('R2: ' + str(round(r2_calculator(lgbm_model.predict(X_valid),y_valid),4)))
print('Adjusted R2: ' + str(round(adjusted_r2_calculator(lgbm_model.predict(X_valid),y_valid),4)))
print('MAE: ' + str(round(mae_and_mse_calculator(lgbm_model.predict(X_valid),y_valid)[0],4)))
print('RMSE: ' + str(round(mae_and_mse_calculator(lgbm_model.predict(X_valid),y_valid)[1]**0.5,4)))

Cosine Similarity Validation Set
R2: 0.9091
Adjusted R2: 0.9091
MAE: 0.0688
RMSE: 0.1107


When the hyperparameter search completes, the result indicates that the error is irreducible. Two ways to reduce this error are feature engineering and data wrangling. Besides this, the random forest model is top choice for this problem because this model is more resistant to overfitting. 

In [24]:
print('Cos Similarity Testing Set')
print('R2: ' + str(round(r2_calculator(rfr_model.predict(X_test),y_test),4)))
print('Adjusted R2: ' + str(round(adjusted_r2_calculator(rfr_model.predict(X_test),y_test),4)))
print('MAE: ' + str(round(mae_and_mse_calculator(rfr_model.predict(X_test),y_test)[0],4)))
print('RMSE: ' + str(round(mae_and_mse_calculator(rfr_model.predict(X_test),y_test)[1]**0.5,4)))

Cos Similarity Testing Set
R2: 0.9097
Adjusted R2: 0.9097
MAE: 0.0686
RMSE: 0.1106


In [25]:
print(f'Time taken to run: {time.time() - start} seconds')

Time taken to run: 77.93327593803406 seconds
